In [1]:
import os
import torch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Self Attention and Masked Self Attention

In [3]:
class Attention(nn.Module):

    def __init__(self, d_model=2, row_dim=0, col_dim=1):
        super().__init__()

        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = row_dim
        self.col_dim = col_dim

    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
        q = self.W_q(encodings_for_q)
        k = self.W_k(encodings_for_k)
        v = self.W_v(encodings_for_v)


        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))
        scaled_sims = sims / torch.tensor(k.size(self.col_dim) ** 0.5)
        if mask is not None:
            scaled_sims = sims / torch.tensor(k.size(self.col_dim) ** 0.5)
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
        attention_score = torch.matmul(attention_percents, v)
        return attention_score

### Encoder - Decoder Attention

In [4]:
encodings_for_q = torch.tensor([
    [1.16, 0.23],
    [0.57, 1.36],
    [4.41, -2.16]
])

encodings_for_k = torch.tensor([
    [1.16, 0.23],
    [0.57, 1.36],
    [4.41, -2.16]
])

encodings_for_v = torch.tensor([
    [1.16, 0.23],
    [0.57, 1.36],
    [4.41, -2.16]
])



torch.manual_seed(42)


attention = Attention(d_model=2, row_dim=0, col_dim=1)

attention(encodings_for_q, encodings_for_k, encodings_for_v)

tensor([[1.0100, 1.0641],
        [0.2040, 0.7057],
        [3.4989, 2.2427]], grad_fn=<MmBackward0>)

# Multi-Head Attention

In [5]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model=2, row_dim=0, col_dim=1, num_heads=1):
        super().__init__()

        self.heads = nn.ModuleList(
            [Attention(d_model, row_dim, col_dim) for _ in range(num_heads)]
        )

        self.col_dim = col_dim

    
    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v):

        return torch.cat(
            [head(encodings_for_q, encodings_for_k, encodings_for_v) for head in self.heads]
            , dim=self.col_dim
        )

### Calculate Multi-Head Attention

In [6]:
torch.manual_seed(42)

multi_head_attention = MultiHeadAttention(d_model=2, row_dim=0, col_dim=1, num_heads=2)

multi_head_attention(encodings_for_q, encodings_for_k, encodings_for_v)

tensor([[ 1.0100,  1.0641, -0.7081, -0.8268],
        [ 0.2040,  0.7057, -0.7417, -0.9193],
        [ 3.4989,  2.2427, -0.7190, -0.8447]], grad_fn=<CatBackward0>)

In [7]:
torch.manual_seed(42)

multi_head_attention = MultiHeadAttention(d_model=2, row_dim=0, col_dim=1, num_heads=3)

multi_head_attention(encodings_for_q, encodings_for_k, encodings_for_v)

tensor([[ 1.0100,  1.0641, -0.7081, -0.8268,  0.6226,  0.1312],
        [ 0.2040,  0.7057, -0.7417, -0.9193,  0.5522,  0.2499],
        [ 3.4989,  2.2427, -0.7190, -0.8447,  0.5669,  0.2324]],
       grad_fn=<CatBackward0>)